## 1. Import Libraries & Configuration

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Color palette for consistent styling
COLORS = {
    'primary': '#0d6efd',
    'success': '#28a745',
    'warning': '#ffc107',
    'danger': '#dc3545',
    'info': '#17a2b8',
    'campaigns': ['#0d6efd', '#6610f2', '#6f42c1', '#d63384', '#fd7e14']
}

print('✅ Libraries imported successfully!')
print(f'📅 Analysis Date: {datetime.now().strftime("%Y-%m-%d %H:%M")}')

## 2. Load All Datasets

In [ ]:
# Define data paths
DATA_PATH = '../data/'

# Load all 4 CSV files
marketing_spend = pd.read_csv(f'{DATA_PATH}marketing_spend.csv')
funnel_events = pd.read_csv(f'{DATA_PATH}funnel_events.csv')
revenue_marketing = pd.read_csv(f'{DATA_PATH}revenue_marketing.csv')
revenue_finance = pd.read_csv(f'{DATA_PATH}revenue_finance.csv')

# Store in dictionary for easy iteration
datasets = {
    'Marketing Spend': marketing_spend,
    'Funnel Events': funnel_events,
    'Revenue (Marketing)': revenue_marketing,
    'Revenue (Finance)': revenue_finance
}

print('✅ All datasets loaded successfully!\n')
print('=' * 60)

## 3. Dataset Shapes & Overview

In [ ]:
# Print shapes for all datasets
print('📋 DATASET DIMENSIONS\n')
print(f'{"Dataset":<25} {"Rows":>8} {"Columns":>10}')
print('-' * 45)

shape_data = []
for name, df in datasets.items():
    print(f'{name:<25} {df.shape[0]:>8,} {df.shape[1]:>10}')
    shape_data.append({'Dataset': name, 'Rows': df.shape[0], 'Columns': df.shape[1]})

print('-' * 45)
print(f'{"TOTAL":<25} {sum(d["Rows"] for d in shape_data):>8,}')

# Create summary DataFrame
shape_summary = pd.DataFrame(shape_data)
shape_summary

## 4. First 5 Rows of Each Dataset

In [ ]:
# Marketing Spend - first 5 rows
print('📊 MARKETING SPEND (First 5 Rows)')
print('Campaign-level daily spend data\n')
marketing_spend.head()

In [ ]:
# Funnel Events - first 5 rows
print('🔄 FUNNEL EVENTS (First 5 Rows)')
print('User journey events with timestamps\n')
funnel_events.head()

In [ ]:
# Revenue Marketing - first 5 rows
print('💰 REVENUE - MARKETING TEAM (First 5 Rows)')
print('Marketing-attributed revenue by campaign\n')
revenue_marketing.head()

In [ ]:
# Revenue Finance - first 5 rows
print('🏦 REVENUE - FINANCE TEAM (First 5 Rows)')
print('Finance-verified revenue by product\n')
revenue_finance.head()

## 5. Statistical Summary (describe())

In [ ]:
# Marketing Spend statistics
print('📊 MARKETING SPEND - Statistical Summary\n')
marketing_spend.describe()

In [ ]:
# Funnel Events statistics
print('🔄 FUNNEL EVENTS - Statistical Summary\n')
funnel_events.describe()

In [ ]:
# Revenue comparison statistics
print('💰 REVENUE COMPARISON - Statistical Summary\n')
revenue_stats = pd.DataFrame({
    'Marketing': revenue_marketing['revenue'].describe(),
    'Finance': revenue_finance['revenue'].describe()
})
revenue_stats

## 6. Missing Values Analysis

In [ ]:
# Check missing values for all datasets
print('🔍 MISSING VALUES ANALYSIS\n')
print('=' * 60)

for name, df in datasets.items():
    missing = df.isnull().sum()
    missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
    
    print(f'\n📌 {name}')
    print('-' * 40)
    
    if missing.sum() == 0:
        print('✅ No missing values!')
    else:
        missing_df = pd.DataFrame({
            'Missing Count': missing,
            'Missing %': missing_pct
        })
        print(missing_df[missing_df['Missing Count'] > 0])

print('\n' + '=' * 60)
print('✅ Missing value analysis complete!')

## 7. Data Type Conversion

In [ ]:
# Convert date columns to datetime
marketing_spend['date'] = pd.to_datetime(marketing_spend['date'])
revenue_marketing['date'] = pd.to_datetime(revenue_marketing['date'])
revenue_finance['date'] = pd.to_datetime(revenue_finance['date'])
funnel_events['timestamp'] = pd.to_datetime(funnel_events['timestamp'])

# Add date column to funnel events
funnel_events['date'] = funnel_events['timestamp'].dt.date

print('✅ Date columns converted to datetime!')
print('\nData Types After Conversion:')
print('-' * 40)
for name, df in datasets.items():
    print(f'\n{name}:')
    print(df.dtypes)

---
# 📈 VISUALIZATIONS
---

## 8. Spend by Channel - Pie Chart

In [ ]:
# Aggregate spend by campaign
spend_by_channel = marketing_spend.groupby('campaign')['spend'].sum().sort_values(ascending=False)

# Create pie chart
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie Chart
colors = COLORS['campaigns'][:len(spend_by_channel)]
explode = [0.02] * len(spend_by_channel)

wedges, texts, autotexts = axes[0].pie(
    spend_by_channel.values,
    labels=spend_by_channel.index,
    autopct='%1.1f%%',
    colors=colors,
    explode=explode,
    shadow=True,
    startangle=90
)

axes[0].set_title('Marketing Spend Distribution by Channel', fontsize=14, fontweight='bold')

# Bar Chart for comparison
bars = axes[1].barh(spend_by_channel.index, spend_by_channel.values, color=colors)
axes[1].set_xlabel('Total Spend ($)')
axes[1].set_title('Spend by Channel (Absolute Values)', fontsize=14, fontweight='bold')

# Add value labels on bars
for bar, value in zip(bars, spend_by_channel.values):
    axes[1].text(value + 100, bar.get_y() + bar.get_height()/2, 
                 f'${value:,.0f}', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('../reports/spend_by_channel.png', dpi=150, bbox_inches='tight')
plt.show()

# Print summary table
print('\n📊 Spend by Channel Summary:')
spend_summary = pd.DataFrame({
    'Total Spend': spend_by_channel,
    'Percentage': (spend_by_channel / spend_by_channel.sum() * 100).round(1)
})
spend_summary['Percentage'] = spend_summary['Percentage'].astype(str) + '%'
spend_summary

## 9. Funnel Event Timeline

In [ ]:
# Aggregate events by date and type
events_timeline = funnel_events.groupby(['date', 'event_type']).size().unstack(fill_value=0)

# Define event order for proper stacking
event_order = ['page_view', 'add_to_cart', 'checkout', 'purchase']
event_order = [e for e in event_order if e in events_timeline.columns]
events_timeline = events_timeline[event_order]

# Create stacked area chart
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Stacked Area Chart
colors = ['#0d6efd', '#6f42c1', '#fd7e14', '#28a745']
events_timeline.plot(
    kind='area',
    stacked=True,
    ax=axes[0],
    color=colors[:len(event_order)],
    alpha=0.7
)

axes[0].set_title('Funnel Events Over Time (Stacked Area)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Number of Events')
axes[0].legend(title='Event Type', loc='upper right')

# Line Chart (individual trends)
for i, col in enumerate(event_order):
    axes[1].plot(events_timeline.index, events_timeline[col], 
                 marker='o', label=col, color=colors[i], linewidth=2)

axes[1].set_title('Funnel Events Over Time (Individual Lines)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Number of Events')
axes[1].legend(title='Event Type', loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/funnel_timeline.png', dpi=150, bbox_inches='tight')
plt.show()

# Print event counts
print('\n🔄 Event Type Counts:')
event_counts = funnel_events['event_type'].value_counts()
event_counts

## 10. Revenue Comparison: Marketing vs Finance

In [ ]:
# Aggregate daily revenue from both sources
mkt_daily = revenue_marketing.groupby('date')['revenue'].sum().reset_index()
mkt_daily.columns = ['date', 'marketing_revenue']

fin_daily = revenue_finance.groupby('date')['revenue'].sum().reset_index()
fin_daily.columns = ['date', 'finance_revenue']

# Merge on date
revenue_comparison = pd.merge(mkt_daily, fin_daily, on='date', how='outer').fillna(0)
revenue_comparison = revenue_comparison.sort_values('date')

# Calculate variance
revenue_comparison['variance'] = revenue_comparison['marketing_revenue'] - revenue_comparison['finance_revenue']
revenue_comparison['variance_pct'] = (
    revenue_comparison['variance'] / revenue_comparison['finance_revenue'].replace(0, np.nan) * 100
).fillna(0)

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Line comparison
axes[0, 0].plot(revenue_comparison['date'], revenue_comparison['marketing_revenue'], 
                label='Marketing', color=COLORS['primary'], linewidth=2, marker='o')
axes[0, 0].plot(revenue_comparison['date'], revenue_comparison['finance_revenue'], 
                label='Finance', color=COLORS['success'], linewidth=2, marker='s')
axes[0, 0].fill_between(revenue_comparison['date'], 
                        revenue_comparison['marketing_revenue'], 
                        revenue_comparison['finance_revenue'], 
                        alpha=0.3, color=COLORS['danger'])
axes[0, 0].set_title('Daily Revenue: Marketing vs Finance', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Revenue ($)')
axes[0, 0].legend()
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Variance bar chart
colors_variance = ['#dc3545' if x > 0 else '#28a745' for x in revenue_comparison['variance']]
axes[0, 1].bar(revenue_comparison['date'], revenue_comparison['variance'], color=colors_variance)
axes[0, 1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0, 1].set_title('Daily Variance (Marketing - Finance)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Variance ($)')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Total comparison bar
totals = {
    'Marketing': revenue_comparison['marketing_revenue'].sum(),
    'Finance': revenue_comparison['finance_revenue'].sum()
}
bars = axes[1, 0].bar(totals.keys(), totals.values(), color=[COLORS['primary'], COLORS['success']])
axes[1, 0].set_title('Total Revenue Comparison', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Total Revenue ($)')

# Add value labels
for bar in bars:
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                    f'${height:,.0f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# 4. Variance summary metrics
total_mkt = totals['Marketing']
total_fin = totals['Finance']
total_variance = total_mkt - total_fin
variance_pct = (total_variance / total_fin * 100) if total_fin > 0 else 0

summary_text = f"""
REVENUE VARIANCE SUMMARY
{'='*40}

Marketing Reported:  ${total_mkt:>12,.0f}
Finance Verified:    ${total_fin:>12,.0f}
{'─'*40}
VARIANCE:            ${total_variance:>12,.0f}
VARIANCE %:          {variance_pct:>12.1f}%

{'='*40}
⚠️  Marketing is OVER-REPORTING by {variance_pct:.1f}%
"""

axes[1, 1].text(0.1, 0.5, summary_text, transform=axes[1, 1].transAxes,
                fontsize=11, verticalalignment='center', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
axes[1, 1].axis('off')
axes[1, 1].set_title('Key Metrics', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/revenue_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Basic Statistics Summary Table

In [ ]:
# Create comprehensive statistics table
stats_data = []

# Marketing Spend stats
stats_data.append({
    'Dataset': 'Marketing Spend',
    'Metric': 'Total Spend',
    'Value': f"${marketing_spend['spend'].sum():,.0f}",
    'Details': f"{len(marketing_spend)} daily records"
})
stats_data.append({
    'Dataset': 'Marketing Spend',
    'Metric': 'Avg Daily Spend',
    'Value': f"${marketing_spend['spend'].mean():,.0f}",
    'Details': f"Std Dev: ${marketing_spend['spend'].std():,.0f}"
})
stats_data.append({
    'Dataset': 'Marketing Spend',
    'Metric': 'Campaigns',
    'Value': str(marketing_spend['campaign'].nunique()),
    'Details': ', '.join(marketing_spend['campaign'].unique())
})

# Funnel Events stats
stats_data.append({
    'Dataset': 'Funnel Events',
    'Metric': 'Total Events',
    'Value': f"{len(funnel_events):,}",
    'Details': f"{funnel_events['event_type'].nunique()} event types"
})
stats_data.append({
    'Dataset': 'Funnel Events',
    'Metric': 'Unique Users',
    'Value': str(funnel_events['user_id'].nunique()),
    'Details': 'Across all events'
})
checkout_users = funnel_events[funnel_events['event_type'] == 'checkout']['user_id'].nunique()
stats_data.append({
    'Dataset': 'Funnel Events',
    'Metric': 'Converted Users',
    'Value': str(checkout_users),
    'Details': 'Users with checkout event'
})

# Revenue stats
stats_data.append({
    'Dataset': 'Revenue (Marketing)',
    'Metric': 'Total Revenue',
    'Value': f"${revenue_marketing['revenue'].sum():,.0f}",
    'Details': f"{len(revenue_marketing)} records"
})
stats_data.append({
    'Dataset': 'Revenue (Finance)',
    'Metric': 'Total Revenue',
    'Value': f"${revenue_finance['revenue'].sum():,.0f}",
    'Details': f"{len(revenue_finance)} records"
})

# Variance
variance = revenue_marketing['revenue'].sum() - revenue_finance['revenue'].sum()
variance_pct = variance / revenue_finance['revenue'].sum() * 100
stats_data.append({
    'Dataset': 'VARIANCE',
    'Metric': 'Revenue Gap',
    'Value': f"${variance:,.0f}",
    'Details': f'{variance_pct:.1f}% over-reporting'
})

# Create DataFrame
stats_table = pd.DataFrame(stats_data)

# Display styled table
print('📊 COMPREHENSIVE STATISTICS SUMMARY\n')
print('=' * 80)
stats_table

In [ ]:
# Create styled HTML table for export
styled_table = stats_table.style.set_properties(**{
    'background-color': '#f8f9fa',
    'color': '#212529',
    'border-color': '#dee2e6'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#0d6efd'), ('color', 'white')]}
])

# Save to HTML
styled_table.to_html('../reports/stats_summary.html')
print('✅ Statistics table exported to reports/stats_summary.html')

styled_table

---
## 12. Key Insights Summary

In [ ]:
# Print key insights
print('═' * 70)
print('                       KEY INSIGHTS SUMMARY                          ')
print('═' * 70)

# Calculate key metrics
total_spend = marketing_spend['spend'].sum()
mkt_revenue = revenue_marketing['revenue'].sum()
fin_revenue = revenue_finance['revenue'].sum()
variance = mkt_revenue - fin_revenue
variance_pct = (variance / fin_revenue * 100)
mkt_roas = mkt_revenue / total_spend
fin_roas = fin_revenue / total_spend

total_users = funnel_events['user_id'].nunique()
checkout_users = funnel_events[funnel_events['event_type'] == 'checkout']['user_id'].nunique()
conversion_rate = (checkout_users / total_users * 100)

insights = f"""
📌 FINDING #1: Revenue Discrepancy
   • Marketing reports ${mkt_revenue:,.0f} in revenue
   • Finance verifies only ${fin_revenue:,.0f}
   • Gap: ${variance:,.0f} ({variance_pct:.1f}% over-reporting)

📌 FINDING #2: ROAS Analysis
   • Marketing ROAS: {mkt_roas:.2f}x (based on reported revenue)
   • Finance-adjusted ROAS: {fin_roas:.2f}x (based on actuals)
   • Difference: {mkt_roas - fin_roas:.2f}x overstated

📌 FINDING #3: Funnel Performance
   • Total unique users: {total_users}
   • Converted users (checkout): {checkout_users}
   • Overall conversion rate: {conversion_rate:.1f}%

📌 FINDING #4: Campaign Distribution
   • Total marketing spend: ${total_spend:,.0f}
   • Number of campaigns: {marketing_spend['campaign'].nunique()}
   • Average daily spend: ${marketing_spend['spend'].mean():,.0f}

═══════════════════════════════════════════════════════════════════════
⚠️  RECOMMENDATION: Investigate attribution methodology differences
    between Marketing and Finance teams to resolve the {variance_pct:.0f}% variance.
═══════════════════════════════════════════════════════════════════════
"""

print(insights)

---
## ✅ EDA Complete

### Next Steps:
1. Deep-dive into spend analysis by campaign (`02_spend_analysis.ipynb`)
2. Revenue reconciliation at granular level (`03_revenue_reconciliation.ipynb`)
3. Funnel optimization analysis (`04_funnel_analysis.ipynb`)

### Outputs Generated:
- `reports/spend_by_channel.png`
- `reports/funnel_timeline.png`
- `reports/revenue_comparison.png`
- `reports/stats_summary.html`